In [1]:
import pickle

movie = pickle.load( open( "movie_tweet_2.pkl", "rb" ) )

In [2]:
from copy import copy

movie_tweet = copy(movie)

In [3]:
import re

for idx, data in enumerate(movie_tweet):
    title = data['title'].lower()
    movie_conversation = []
    for tweet in data['tweet']:
        cleaned_tweet = re.sub('[^A-Za-z0-9_\s]', "", tweet.lower())
        tweet_list = cleaned_tweet.split(" ")
        if title in tweet_list:
            tweet_list.remove(title)
        if '' in tweet_list:
            space_count = tweet_list.count('')
            for _ in range(space_count):
                tweet_list.remove('')
    
        movie_conversation.append(' '.join(tweet_list))
    movie_tweet[idx]['tweet'] = movie_conversation

In [4]:
from nltk.tokenize import word_tokenize
# from nltk.tag import pos_tag
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

st = PorterStemmer()
stop = set(stopwords.words('english'))

tweet_token = copy(movie_tweet)

for idx, data in enumerate(movie_tweet):
    tweet_data = []
    for tweet in data['tweet']:
#         tweet_data.append(pos_tag(word_tokenize(tweet)))
        token_and_stem = [st.stem(i) for i in word_tokenize(tweet)]
        filtered_words = [i for i in token_and_stem if i not in stop]    
        
        tweet_data.append(filtered_words)
        
    tweet_token[idx]['tweet'] = tweet_data

In [5]:
import pandas as pd

df = pd.DataFrame.from_dict(tweet_token)

In [6]:
vocab = []
for i in df.tweet:
    for j in i:
        vocab += j

In [7]:
import collections

#print(collections.Counter(vocab).most_common(3000), "\n")
most_common_vocab = collections.Counter(vocab).most_common(3000)

_vocab = [word for word, common in most_common_vocab]

unknown_token = 'UNKNOWN_TOKEN'
_vocab.append(unknown_token)

In [8]:
vocab_dict = {word:idx for idx, word in enumerate(_vocab)}

In [9]:
index_to_word = copy(df.tweet)

index_to_word[0]

[['made', 'ref', 'help', 'peopl', 'draw', 'fireemblemhero'],
 ['ref', 'use', 'make', 'tutori'],
 ['goddess', 'thank', 'yiu', 'much'],
 ['np', 'gotchu', '3']]

In [10]:
for row, thread in enumerate(index_to_word):
    for num ,tweet in enumerate(thread):
        conversation_list = []
        for idx, word in enumerate(tweet):
#             one_hot_tweet  = [0 for _ in range(len(vocab_dict))]
#             print(vocab_dict[word])
#             one_hot_tweet[vocab_dict[word]] = 1
#         conversation_list.append(one_hot_tweet)
            if word in vocab_dict.keys():
                index_to_word[row][num][idx] = vocab_dict[word]
            else:
                index_to_word[row][num][idx] = vocab_dict[unknown_token]

In [11]:
title_index = copy(df.title)

In [12]:
title_dict = {title:idx for idx, title in enumerate(set(title_index))}

In [13]:
for idx, title in enumerate(title_index):
    title_index[idx] = title_dict[title]

In [14]:
for a, b in title_dict.items():
    if b==1536:
        print(a)

Tsotsi


In [15]:
len(title_dict)

1847

In [16]:
len(title_index)

7919

In [17]:
max_word_size = 0
max_conversation_size = 0

for conversation_list in index_to_word:
    if len(conversation_list) > max_conversation_size:
        max_conversation_size = len(conversation_list)
    for word_list in conversation_list:
        if len(word_list) > max_word_size:
            max_word_size = len(word_list)

In [18]:
len(index_to_word), max_conversation_size, max_word_size

(7919, 9, 27)

In [19]:
DEFAULT_VALUE = 10000

for conversation_idx, conversation_list in enumerate(index_to_word):
    if len(conversation_list) < max_conversation_size:
        default_list_size = max_conversation_size-len(conversation_list)
        for _ in range(default_list_size):
            default_list = [DEFAULT_VALUE for i in range(max_word_size)]
            index_to_word[conversation_idx].append(default_list)
            
    for word_list_idx, word_list in enumerate(conversation_list):
        if len(word_list) < max_word_size:
            current_word_list_size = len(word_list)
            empty_word_space_size = max_word_size - current_word_list_size
            for _ in range(empty_word_space_size):
                index_to_word[conversation_idx][word_list_idx].append(DEFAULT_VALUE)

In [20]:
import tensorflow as tf
import numpy as np

ImportError: Traceback (most recent call last):
  File "/home/hyeonju/anaconda3/envs/ml_python/lib/python3.4/site-packages/tensorflow/python/__init__.py", line 61, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "/home/hyeonju/anaconda3/envs/ml_python/lib/python3.4/site-packages/tensorflow/python/pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "/home/hyeonju/anaconda3/envs/ml_python/lib/python3.4/site-packages/tensorflow/python/pywrap_tensorflow.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow', fp, pathname, description)
  File "/home/hyeonju/anaconda3/envs/ml_python/lib/python3.4/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
ImportError: libcudart.so.8.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/g3doc/get_started/o

In [21]:
X = tf.placeholder('float', shape = [None, max_conversation_size, max_word_size])

In [22]:
x_data = tf.reshape(X, [-1, max_conversation_size, max_word_size, 1])

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])